In [91]:
import re
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# company data 가져오기
with open('./data/company.json') as f:
     data = json.loads(f.read())

# 필요한 데이터로만 딕셔너리 생성
company_loc = {item['id']: {'name':item['name'], 'address':item['location']['address'], 'geo_lat': item['location']['geo_lat'], 'geo_alt': item['location']['geo_alt']} for item in data}

print(company_loc)

{'13138': {'name': '프리그로우', 'address': '부산시 부산진구 양연로4 2층', 'geo_lat': 35.1751860797821, 'geo_alt': 129.078568052224}, '13107': {'name': '비욘드메디슨', 'address': '서울 강남구 역삼동 824-12 메가시티오피스텔 403호', 'geo_lat': 37.4973947147549, 'geo_alt': 127.030232215583}, '13104': {'name': '언리저브', 'address': '서울특별시 강남구 논현동 논현로 123길 36', 'geo_lat': 37.5109939153304, 'geo_alt': 127.028933358994}, '13005': {'name': '에스피에이디', 'address': '서울 성북구 안암로 145, (안암동5가)', 'geo_lat': 37.5887034223667, 'geo_alt': 127.031698331241}, '12939': {'name': '언컷젬스컴퍼니', 'address': '서울 강남구 강남대로152길 20, (신사동) 5층', 'geo_lat': 37.5173922520879, 'geo_alt': 127.021098483702}, '12675': {'name': '와드', 'address': '경기 성남시 분당구 대왕판교로 660, (삼평동) 유스페이스1 A동 605호', 'geo_lat': 37.4004659332759, 'geo_alt': 127.106812648944}, '12647': {'name': '메이즈', 'address': '서울 구로구 디지털로31길 20, (구로동)', 'geo_lat': 37.4855210244653, 'geo_alt': 126.894345286511}, '12644': {'name': '제이에스소프트', 'address': '서울 영등포구 문래로20길 60, (문래동3가) 702호', 'geo_lat': 37.5172701562336, '

In [92]:
# 주소에서 '동' 추출
df['dong'] = df['address'].apply(lambda x: re.search(r'\b[가-힣]*동\b', x).group() if re.search(r'\b[가-힣]*동\b', x) else None)

# 동별 개수 세기
df_dong = df['dong'].value_counts().reset_index()

df_dong.columns = ['dong', 'count']


print(df_dong)


    dong  count
0    역삼동     18
1    서초동      8
2    논현동      7
3    삼평동      6
4    삼성동      6
5    구로동      4
6    가산동      3
7    대치동      3
8    도곡동      3
9    공덕동      3
10  여의도동      2
11   반포동      2
12   동천동      2
13   신사동      2
14   양재동      2
15   합정동      2
16   포일동      1
17   문지동      1
18   삼락동      1
19   구미동      1
20   창곡동      1
21   서현동      1
22   상암동      1
23   가양동      1
24   마곡동      1
25   시흥동      1
26    다동      1
27   성내동      1
28   상매동      1
29   신내동      1
30   성산동      1
31   정자동      1
32   호계동      1
33   금토동      1
34   관철동      1
35   신천동      1
36   석우동      1
37   문정동      1
38   도화동      1
39   풍산동      1
40   동자동      1
41   상수동      1
42    우동      1
43   신흥동      1
44   송파동      1
45   고색동      1
46   서교동      1
47   잠실동      1


In [93]:
# 동 기준으로 위도와 경도의 평균 계산
df_dong_mean = df.groupby('dong').agg({'geo_lat':'mean', 'geo_alt':'mean'}).reset_index()

# 동별 개수와 평균 위도, 경도를 결합
df_final = pd.merge(df_dong, df_dong_mean, on='dong')

# 결과 출력
print(df_final)

    dong  count    geo_lat     geo_alt
0    역삼동     18  37.499841  127.034917
1    서초동      8  37.492720  127.021778
2    논현동      7  37.511975  127.030917
3    삼평동      6  37.399634  127.109235
4    삼성동      6  37.508863  127.053641
5    구로동      4  37.483840  126.893792
6    가산동      3  37.475723  126.885064
7    대치동      3  37.505068  127.053876
8    도곡동      3  37.490674  127.036681
9    공덕동      3  37.546086  126.951899
10  여의도동      2  37.522625  126.927823
11   반포동      2  37.507270  127.021941
12   동천동      2  37.338255  127.101362
13   신사동      2  37.518441  127.023869
14   양재동      2  37.475424  127.041835
15   합정동      2  37.547093  126.916652
16   포일동      1  37.401015  126.991356
17   문지동      1  36.387183  127.404096
18   삼락동      1  35.191570  128.983712
19   구미동      1  37.340028  127.107815
20   창곡동      1  37.465597  127.136705
21   서현동      1  37.387582  127.123716
22   상암동      1  37.579428  126.889783
23   가양동      1  37.570717  126.841131
24   마곡동      1  37.55847

In [94]:
# 회사 데이터 df변환
df = pd.DataFrame.from_dict(company_loc, orient='index')

# 맵 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=10)

# 맵에 버블 추가
for idx, row in df.iterrows():
    folium.CircleMarker(location=[row['geo_lat'], row['geo_alt']],
                        fill=True).add_to(m)

# m.save('map.html')

m

In [95]:
# 서울 중심부의 위도, 경도
map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

# 맵에 회사 위치 추가
for idx, row in df.iterrows():
    # 각 회사의 위도, 경도
    folium.CircleMarker(location=[row['geo_lat'], row['geo_alt']],
                        fill=True,
                        color='red').add_to(map)

# 동 별로 회사의 개수 표시
# MarkerCluster 인스턴스 생성
marker_cluster = MarkerCluster().add_to(map)

for idx, row in df_final.iterrows():
    # 각 동의 위도, 경도
    folium.Marker(location=[row['geo_lat'], row['geo_alt']], 
                  popup=row['dong'], 
                  icon=folium.Icon(color='blue', icon='ok-sign')).add_to(marker_cluster)

# map.save('map.html')
map